In [1]:
%matplotlib widget
from pylab import axes, setp, axis, xlabel, ylabel, title, plot, subplot, draw, connect, show
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eig
import matplotlib.pyplot as plt


def potential(x, mu):
    """Potentialfunktion fuer  Doppelmulde, mu: Parameter"""
    if ptype == 'doppelmulde':
        return x**4 - x**2 + mu*x # asymmetrische Doppelmulde


def singlewell_potential(x, x0, width, height):
    x1 = zeros(len(x))
    for i in range(len(x)):
        if x[i] > x0 - width/2.0 and x[i] < x0 + width/2.0:
            x1[i] = height
    return x1
    

def diagonalisierung(hquer, L, N, pot=singlewell_potential, x0 = 0.0, width = 0.1, height = 0.0):
    """Berechne sortierte Eigenwerte und zugehoerige Eigenfunktionen.

       Eingabe:
         hquer: effektives hquer
         L: legt betrachtetes Intervall [-L,L] fest
         N: Zahl der Gitterpunkte, d.h. Groesse der Matrix
         pot: Potentialfunktion der Form pot(x, mu)
         mu: Potentialparameter
       Ausgabe:
         ew: sortierte Eigenwerte (Array der Laenge N)
         ef: entsprechend sortierte Eigenvektoren, ef[:,i] (Groesse N*N)
         x: Ortsgitter (Array der Laenge N)
         dx: Ortsgitterabstand
         V: Potential an den Stellen x (Array der Laenge N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # Gitterpunkte 
    dx = x[1] - x[0]                              # Gitterabstand
    V = pot(x, x0, width, height)
    z = hquer**2 /2.0/dx**2                       # Nebendiagonalen
    h = (diag(V+2.0*z) + diag(-z*ones(N-1), -1)   # Aufstellen der
                      + diag(-z*ones(N-1), 1) )   #  Hamilton-Matrix

    ew, ef = eig(h)                               # Diagonalisierung
    ew = ew.real                                  # Realteil der Eigenwerte
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sortieren der ew nach ind
    ef = ef[:, ind]                               # Sortieren der Spalten von
                                                  #  ef nach ind
    ef = ef/sqrt(dx)                              # richtige Normierung 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.1, fak=0.009):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    
    ax[0].plot(x, V, c='r')
    ax[1].plot(x, V, c='k', linewidth=1.0)
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V), Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V), Emax])
    
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
      
    indmax = sum(ew<=Emax)                       
    colors = ['b', 'g', 'r', 'c', 'm', 'y']      
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[1].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])

    
#     fig = plt.figure()
    
#     ax1 = fig.add_subplot(211, xlim=(min(x), max(x)), ylim = (min(V), Emax))
#     line1 = ax1.plot(x, V, c='r', figsize=(20, 20))
    
#     ax2 = fig.add_subplot(221, xlim=(min(x), max(x)), ylim = (min(V), Emax))
#     line2 = ax2.plot(x, V, c='r', figsize=(20, 20))
#     plt.show()
    
#     fig = plt.figure(1, title = "Asymm. Double Well Potential")
#     axes_options = {'x': {'label': r'$x/a$'}, 'y': {'label': r'$V(x)/V_0$'}}
#     lv = plt.plot(x, V, axes_options = axes_options, colors = ['black'])
#     plt.xlim(min(x), max(x))
#     plt.ylim(min(V), Emax)

#     indmax = sum(ew<=Emax)                       # Zahl der zu plottenden Efkt
#     colors = ['b', 'g', 'r', 'c', 'm', 'y']      # sorgt fuer richtige Farben
#     if not hasattr(width, "__iter__"):           # Falls `width` skalar:
#         width = width*ones(indmax)               #   konst. Linienstaerke
#     for i in arange(indmax):                     # Plot der Eigenfunktionen
#         plt.plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1)
    
#     lv.on_background_click(lambda self, x: print(x))
    
#     axes(ax)                                     # Ortsraumplotfenster
#     setp(ax, autoscale_on=False)
#     axis([min(x), max(x), min(V), Emax]) 
#     xlabel(r'$x/a$')
#     ylabel(r'$V(x)/V_0\ \rm{,\ Eigenfunktionen\ bei\ Eigenwert}$')
#     title("Asymm. Doppelmuldenpotential")
    
#     plot(x, V, linewidth=1, color='.7')          # Potential plotten    
#     indmax = sum(ew<=Emax)                       # Zahl der zu plottenden Efkt
#     colors = ['b', 'g', 'r', 'c', 'm', 'y']      # sorgt fuer richtige Farben
#     if not hasattr(width, "__iter__"):           # Falls `width` skalar:
#         width = width*ones(indmax)               #   konst. Linienstaerke
#     for i in arange(indmax):                     # Plot der Eigenfunktionen
#         plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1,
#                                           color=colors[i%len(colors)])
#

In [2]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

mu = 0.06                                            # Potentialparameter
L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.


swidth = FloatSlider(value = 0.1, min = 0.1, max = 1.0, description = 'Width: ')
sheight = FloatSlider(value = 0.0, min = 0.0, max = 5.0, description = 'Height: ')
sxpos = FloatSlider(value = 0.0, min = -L, max = L, description = r'$x_o$: ')

width = 0.1
height = 0.0

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, x0 = 0.0, width = width, height = height)
                                                     # berechne EW u. Efkt.
    
fig, ax = plt.subplots(1, 2, figsize=(20, 8))

fig.suptitle('Numerial Solution of One Dimension Schroedinger Equation', fontsize = 10)
plot_eigenfunktionen(ax, ew, ef, x, V)


def on_width_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, x0 = sxpos.value, width = swidth.value, height = sheight.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)

def on_height_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, x0 = 0.0, width = swidth.value, height = sheight.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_xpos_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, x0 = sxpos.value, width = swidth.value, height = sheight.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
swidth.observe(on_width_change, names = 'value')
sheight.observe(on_height_change, names = 'value')
sxpos.observe(on_xpos_change, names = 'value')

display(HBox([swidth, sheight, sxpos]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>